In [ ]:
from pathlib import Path

import napari
import numpy as np
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter
from skimage.util import invert
from skimage.io import imread

from chromemt_analysis import parse_simulation_zip, fwhm2sigma

## Simulate experimental effects in simulation output 

In [ ]:
# basedir = Path('E:/chromemt_data/')
# basedir = Path('/Volumes/davidh-ssd/chromemt_data/')
basedir = Path('/Users/david/Desktop/chromemt_data/')

# load simulated data
psz, img_simulation = parse_simulation_zip(basedir / 'simulations/AllCVC.zip', 'InactiveCD.txt')
img_simulation = img_simulation.astype(float)

# load real data
img_real = imread(basedir / '49801.tif')

In [ ]:
# sigma zyx roughly estimated via line profiles through isolated densities in EM data
# minimal FWHMs ~2.5px xy, ~4.5px z -> sigma of ~2.0/1.0 to
# fwhm2sigma((4.5, 2.5, 2.5))

sigma_scope_resolution = (2.0, 1.0, 1.0)

# expected molecules per pixel inside structure
expected_molecules = 0.5

# expected molecules per pixel in background
background_expected_molecules = 0.0

# blur before sampling molecule pos
sigma_diffusion = 1.0

# 1) blur to simulate DAB diffusion
img_expected_molecules = gaussian_filter(img_simulation * expected_molecules, sigma_diffusion) + background_expected_molecules
# 2) Poisson process to simulate sparse labelling
img_sampled_molecules = np.random.poisson(img_expected_molecules).astype(float)
# 3) second blur to simulate microscope resolution
img_simulation_final = gaussian_filter(img_sampled_molecules, sigma_scope_resolution)

# alternative: just microscope resolution
img_simulation_justblur = gaussian_filter(img_simulation, sigma_scope_resolution)

In [ ]:
## cut slice from real data with same xy size as simulated

slices = (slice(np.clip(s // 2 - c // 2, 0, s), np.clip(s // 2 + c // 2 + 1, 0, s)) for s, c in zip(img_real.shape, img_simulation.shape))
img_real_center = img_real[tuple(slices)]

img_real_center.shape, img_simulation.shape

In [ ]:
fig, (axs_mask, axs_sim_justblur, axs_sim, axs_real) = plt.subplots(ncols=2, nrows=4, figsize=(15,30))

mid_xy_mask = invert(img_simulation[img_simulation.shape[0]//2])
mid_xz_mask = invert(img_simulation[:,img_simulation.shape[1]//2])

mid_xy_justblur = invert(img_simulation_justblur[img_simulation_justblur.shape[0]//2])
mid_xz_justblur = invert(img_simulation_justblur[:,img_simulation_justblur.shape[1]//2])

mid_xy = invert(img_simulation_final[img_simulation_final.shape[0]//2])
mid_xz = invert(img_simulation_final[:,img_simulation_final.shape[1]//2])

mid_xy_real = img_real_center[img_real_center.shape[0]//2]
mid_xz_real = img_real_center[:,img_real_center.shape[1]//2]

axs_mask[0].imshow(mid_xy_mask, cmap='gray', clim=(mid_xy_mask.min(), mid_xy_mask.max()*1.1))
axs_mask[0].axis('off')
axs_mask[0].set_title('midplane XY')
axs_mask[1].imshow(mid_xz_mask, cmap='gray', clim=(mid_xz_mask.min(), mid_xz_mask.max()*1.1))
axs_mask[1].axis('off')
axs_mask[1].set_title('midplane XZ')

axs_sim_justblur[0].imshow(mid_xy_justblur, cmap='gray', clim=(mid_xy_justblur.min(), mid_xy_justblur.max()*1.1))
axs_sim_justblur[0].axis('off')
axs_sim_justblur[0].set_title('midplane XY')
axs_sim_justblur[1].imshow(mid_xz_justblur, cmap='gray', clim=(mid_xz_justblur.min(), mid_xz_justblur.max()*1.1))
axs_sim_justblur[1].axis('off')
axs_sim_justblur[1].set_title('midplane XZ')

axs_sim[0].imshow(mid_xy, cmap='gray', clim=(mid_xy.min(), mid_xy.max()*1.1))
axs_sim[0].axis('off')
axs_sim[0].set_title('midplane XY')
axs_sim[1].imshow(mid_xz, cmap='gray', clim=(mid_xz.min(), mid_xz.max()*1.1))
axs_sim[1].axis('off')
axs_sim[1].set_title('midplane XZ')

axs_real[0].imshow(mid_xy_real, cmap='gray')
axs_real[0].axis('off')
axs_real[0].set_title('midplane XY')
axs_real[1].imshow(mid_xz_real, cmap='gray')
axs_real[1].axis('off')
axs_real[1].set_title('midplane XZ')

plt.rc('pdf', fonttype='42')
# plt.savefig('/Users/david/Desktop/density-figures/experimental_effects_reg415.pdf')

In [ ]:
## View simulated image in napari

if napari.current_viewer() is not None:
    napari.current_viewer().close()

viewer = napari.view_image(invert(img_simulation_final))